# 一、Teacher 模型

## 1. 蒸馏模型 Trainer

In [2]:
from unsloth import FastLanguageModel
import torch

from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-12-13 10:15:04.799863: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765620905.200038      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765620905.370839      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
# 计算反向kl散度
def compute_rkl(logits, teacher_logits, target, padding_id, reduction="sum", temp=1.0):
    logits = logits / temp
    teacher_logits = teacher_logits / temp
    probs = torch.softmax(logits, -1, dtype=torch.float32)
    log_probs = torch.log_softmax(logits, -1, dtype=torch.float32)
    teacher_log_probs = torch.log_softmax(teacher_logits, -1,dtype=torch.float32)
    kl = (probs * (log_probs - teacher_log_probs))
    kl = kl.sum(-1)
    
    if reduction == "sum":
        pad_mask = target.eq(padding_id)
        kl = kl.masked_fill_(pad_mask, 0.0)
        kl = kl.sum()
    return kl

In [4]:
from trl import SFTTrainer

class KDTrainer(SFTTrainer):
    def __init__(self, *args, teacher_model=None, if_use_entropy=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.teacher_model = teacher_model
        self.if_use_entropy = if_use_entropy
        
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        outputs_student = model(**inputs)
        
        with torch.no_grad():
            teacher_outputs = self.teacher_model(**inputs)

        # 计算标准的监督损失
        loss = outputs_student.loss
        logits = outputs_student.logits
        
        with torch.no_grad():
            teacher_logits = teacher_outputs.logits
        # 如果教师模型和学生模型输出形状不匹配，对学生模型进行padding或对教师模型进行截断
        # print(logits.shape, teacher_logits.shape)
        # print(type(logits), type(teacher_logits))
        # if logits is None or teacher_logits is None:
        kl = 0
        if isinstance(logits, torch.Tensor) and isinstance(teacher_logits, torch.Tensor):
            # 对齐 logits 维度
            if logits.shape[-1] != teacher_logits.shape[-1]:
                # gap = teacher_logits.shape[-1] - logits.shape[-1]
                # if gap > 0:
                # pad_logits = torch.zeros((logits.shape[0],logits.shape[1], gap)).to(logits.device)
                # logits = torch.cat([logits, pad_logits], dim=-1)
                # 截断 teacher_logits 与学生模型保持一致
                teacher_logits = teacher_logits[:, :, :logits.shape[-1]]
                labels = inputs['labels']
                kl = compute_rkl(logits, teacher_logits, labels, padding_id=-100, temp=2.0)
    
        if self.if_use_entropy:
            loss_total = 0.5 * kl + 0.5 * loss
        else:
            loss_total = kl
        return (loss_total, outputs_student) if return_outputs else loss_total

## 2. 加载 deepseek

In [5]:
max_seq_length = 2048
dtype = None
load_in_4bit = True
teacher, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/kaggle/input/deepseek-teacher-model/deepseek_teacher_finetune",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# FastLanguageModel.for_inference(teacher)

==((====))==  Unsloth 2025.12.5: Fast Qwen2 patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.52G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

Unsloth 2025.12.5 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


# 二、 蒸馏训练

## 1. 加载 Student 模型

In [6]:
# 学生模型
student, _ = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen3-4B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

student = FastLanguageModel.get_peft_model(
    student,
    r = 8, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none", # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False, # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)


student.print_trainable_parameters()

==((====))==  Unsloth 2025.12.5: Fast Qwen3 patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

Unsloth 2025.12.5 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


trainable params: 16,515,072 || all params: 4,038,983,168 || trainable%: 0.4089


## 2. 数据格式化

In [7]:
EOS_TOKEN = tokenizer.eos_token

alpaca_prompt = """You are an expert film critic. Analyze the following review step by step and explain why its sentiment is "{label}".

Review: {input}

<think>
Let's think step by step:
1. Look for emotionally charged words.
2. Assess the overall tone.
3. Justify why the sentiment is "{label}".
</think>"""

def formatting_prompts_func(examples): 
    inputs = examples["sentence"]
    labels = examples["label"]
    outputs = []

    for inp, label in zip(inputs, labels): 
        label_text = "positive" if label == 1 else "negative"
        text = alpaca_prompt.format(input=inp, label=label_text) + EOS_TOKEN 
        outputs.append(text) 
    return {"text": outputs}

## 3. 加载数据

In [8]:
from datasets import load_dataset
import pandas as pd
import datasets


# 加载数据
dataset = load_dataset("sst2")


train_dataset = dataset["train"]
val_dataset = dataset["validation"]
test_dataset = dataset["test"]

tokenized_train = train_dataset.map(formatting_prompts_func, batched=True,)
tokenized_val = val_dataset.map(formatting_prompts_func, batched=True,)

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

## 4. 设置训练参数

In [9]:
# 训练参数
args = TrainingArguments(
    output_dir='results',
    num_train_epochs=1,
    do_train=True,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    logging_steps=100,
    save_strategy='epoch',
    save_total_limit=10,
    fp16 = not is_bfloat16_supported(),
    bf16 = is_bfloat16_supported(),
    learning_rate=0.001,
    lr_scheduler_type='constant',
    optim="adamw_torch_fused",
    report_to = "none", 
)

trainer = KDTrainer(
    model=student,
    teacher_model=teacher,
    if_use_entropy=True,
    tokenizer=tokenizer,
    train_dataset=tokenized_train, 
    eval_dataset=tokenized_val,
    dataset_text_field = "text",
    max_seq_length=max_seq_length,
    dataset_num_proc=1,
    packing=False, # Can make training 5x faster for short sequences.
    args=args,
)

# 如果是初次训练resume_from_checkpoint为false，接着checkpoint继续训练，为True
trainer.train(resume_from_checkpoint=False)

Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/67349 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/872 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 67,349 | Num Epochs = 1 | Total steps = 16,838
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 2 x 1) = 4
 "-____-"     Trainable parameters = 16,515,072 of 4,038,983,168 (0.41% trained)


Step,Training Loss
100,0.941300
200,0.849100
300,0.871300
400,1.725300
500,2.378900
600,2.404700
700,2.297100
800,1.112000
900,0.886900
1000,0.927000


TrainOutput(global_step=16838, training_loss=1.0498971580000611, metrics={'train_runtime': 27114.2685, 'train_samples_per_second': 2.484, 'train_steps_per_second': 0.621, 'total_flos': 1.1925201069300634e+17, 'train_loss': 1.0498971580000611, 'epoch': 1.0})

# 三、 模型预测

In [10]:
from tqdm import tqdm
import numpy as np

def format_test_prompt(examples):
    sentences = examples["sentence"]
    outputs = []
    for sent in sentences:
        text = f"""Analyze the following review and determine if the sentiment is positive or negative.

        Review: {sent}
        
        Sentiment:"""
        outputs.append(text)
    return {"text": outputs}

formatted_test = test_dataset.map(format_test_prompt, batched=True)

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding=False,
        max_length=max_seq_length,
        add_special_tokens=True,
    )

tokenized_test = formatted_test.map(
    tokenize_function,
    batched=True,
    remove_columns=["text", "sentence", "label"],
)


Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [11]:
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd


student.eval()
student.to("cuda")
def parse_prediction(text: str) -> int:
    text = text.strip().lower()
    if "positive" in text:
        return 1
    elif "negative" in text:
        return 0
    if any(word in text for word in ["1", "yes", "good", "great", "excellent", "love", "amazing"]):
        return 1
    if any(word in text for word in ["0", "no", "bad", "awful", "terrible", "hate", "worst"]):
        return 0
    return 0  # default to negative if uncertain
generation_config = {
    "max_new_tokens": 20,
    "do_sample": False,
    "temperature": 0.0,
    "pad_token_id": tokenizer.pad_token_id,
    "eos_token_id": tokenizer.eos_token_id,
}
tokenizer.padding_side = "left"
predictions = []
batch_size = 4
for i in tqdm(range(0, len(tokenized_test), batch_size), desc="Inference"):
    batch = tokenized_test[i:i+batch_size]
    input_ids = [item for item in batch["input_ids"]]
    attention_mask = [item for item in batch["attention_mask"]]

    # 手动 pad（或用 collator）
    max_len = max(len(ids) for ids in input_ids)
    input_ids = [
        [tokenizer.pad_token_id] * (max_len - len(ids)) + ids
        for ids in input_ids
    ]
    attention_mask = [
        [0] * (max_len - len(mask)) + mask
        for mask in attention_mask
    ]

    input_ids = torch.tensor(input_ids).to("cuda")
    attention_mask = torch.tensor(attention_mask).to("cuda")

    with torch.no_grad():
        outputs = student.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=20,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

    for j in range(len(outputs)):
        prompt_start = (input_ids[j] != tokenizer.pad_token_id).nonzero(as_tuple=True)[0][0].item()
        prompt_len = input_ids.shape[1] - prompt_start
        generated_tokens = outputs[j][input_ids.shape[1]:]
        generated = tokenizer.decode(generated_tokens, skip_special_tokens=True)
        pred = parse_prediction(generated)
        predictions.append(pred)

true_labels = np.array(test_dataset["label"])
acc = accuracy_score(true_labels, predictions)

pd.DataFrame({
    "id": range(len(predictions)),
    "sentiment": predictions
}).to_csv("/kaggle/working/decoder_distillation.csv", index=False)

Inference:   0%|          | 0/456 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/unsloth/kernels/utils.py:963: UserWarning: An output with one or more elements was resized since it had shape [1, 4, 2560], which does not match the required output shape [4, 1, 2560]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at /pytorch/aten/src/ATen/native/Resize.cpp:31.)
  out = torch_matmul(X, W.t(), out = out)
/usr/local/lib/python3.11/dist-packages/unsloth/kernels/utils.py:970: UserWarning: An output with one or more elements was resized since it had shape [1, 4, 2560], which does not match the required output shape [4, 1, 2560]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by